# Experiment Tutorial

In [ ]:
from shrinkbench.experiment import PruningExperiment
import numpy as np
import matplotlib.pyplot as plt

The `DATAPATH` environment variable is used to tell the framework where to look for datasets.

In [ ]:
# be sure to change the path
data_path = '/home/zache1/datasets'

In [ ]:
import os
os.environ['DATAPATH'] = data_path


In [ ]:
import torchvision
torchvision.datasets.MNIST(root=data_path + '/MNIST',train=False,download=True)

In [ ]:
from IPython.display import clear_output
clear_output()

We run experiments for our MNIST network for logarithmically spaced compression ratios

In [ ]:
#figure out how to implement my own strategy
from shrinkbench.pruning import (VisionPruning)
import shrinkbench.strategies
from shrinkbench.strategies import utils

class my_strategy(VisionPruning):

    def model_masks(self):
        importances = utils.map_importances(lambda x : np.maximum(0, x), self.params())
        flat_importances = utils.flatten_importances(importances)
        threshold = utils.fraction_threshold(flat_importances, self.fraction)
        masks = utils.importance_masks(importances, threshold)
        return masks

# need to do this so that shrinkbench can get my strategy
setattr(shrinkbench.strategies, 'my_strategy', my_strategy)

for s in ['my_strategy', 'GlobalMagWeight', 'LayerMagWeight']:
    for c in [1,2,4, 8, 16, 32, 64]:
        exp = PruningExperiment(dataset='MNIST', 
                                model='MnistNet',
                                strategy=s,
                                compression=c,
                                train_kwargs={'epochs':10},
                                pretrained=False)
        exp.run()
        clear_output()

We then collect output from experiment folders and plot the diferent metrics easily

In [ ]:
from shrinkbench.plot import df_from_results, plot_df

In [ ]:
df = df_from_results('results')

With the provided functions, it is easy to generate plots 

In [ ]:
plot_df(df, 'compression', 'pre_acc5', markers='strategy', line='--', colors='strategy', suffix=' - pre')
plot_df(df, 'compression', 'post_acc5', markers='strategy', fig=False, colors='strategy')

We can also check the theoretical speedup and see that layerwise provides larger FLOPS speedups because of the even pruning of the conv layers

In [ ]:
plot_df(df, 'speedup', 'post_acc5', colors='strategy', markers='strategy')

#plt.yscale('log')
#plt.ylim(0.985,0.99)

plt.xticks(2**np.arange(7))
plt.gca().set_xticklabels(map(str, 2**np.arange(7)))

None

We can easily check if the compression is matching our expectation by looking at the relative error. As expected, random pruning does worst.

In [ ]:
df['compression_err'] = (df['real_compression'] - df['compression'])/df['compression']

In [ ]:
plot_df(df, 'compression', 'compression_err', colors='strategy', markers='strategy')